In [1]:
%matplotlib inline

# レビューを機械学習できる形にする

## gensim版

1. 単語辞書をつくる
2. 全レビューを分かち書きし、ストップ・ワード処理する
3. 全レビューからコーパスをつくる
4. 各レビューを行列変換する

## HashingTrick版

In [31]:
import re
from gensim import corpora, models, similarities, matutils
import MeCab
import pandas as pd

### レビュー (tsv) を読み込む

In [3]:
csv = pd.read_csv( '~/work/sake/review.tsv', delimiter='\t' )

In [4]:
csv[:5]

,code,meigara,review,datetime
0,4,鰊御殿 山田錦,4合瓶900円で安売りだったので買ってみました。あまり期待してなかったのですが、わりと普通に...,2014-09-14 10:32:46
1,4,北の誉 鰊御殿 純米大吟醸酒,山田錦100％使用。 フルーティな香りと、あっさりした呑み口、 コクのある味わい。 文句の...,2014-05-23 21:35:40
2,4,純米米しぼり,アルコール度数14-15。精米歩合70％。 香りマイルド、深いコク。 オーソドックスな味わ...,2013-08-28 23:19:33
3,4,白ラベル,安いが美味いと思う、ぬる燗か常温がいいね、吟醸酒をブレンドして造ってるて噂があるけどどうでし...,2009-07-07 00:43:14
4,4,純米吟醸『涼しぼり』,近所の酒屋で購入。 「夏のにごり酒」という謳い文句通り、爽やかで呑みやすい。 香りもいいけど...,2008-06-16 23:37:21


In [5]:
mecab = MeCab.Tagger("-Ochasen")

### 1. 単語辞書を作成する

In [6]:
single = r"^[0-9０-９ぁ-んァ-ン！？・　!-/:-@≠\[-`{-~\u3001-\u303F]$"
pair    = r"^[ぁ-んァ-ン]{2}$"

In [7]:
def wakachi( s ):
    res = mecab.parse( s )
    lines = res.split( '\n' )
    words = [ line.split( '\t' )[0] for line in lines ]
    words.remove( 'EOS' )
    words.remove( '' )
    return words

In [8]:
def stop_words_set( ws ):
    return set ( w for w in ws if re.match( single, w ) is None and re.match( pair, w ) is None )

In [9]:
def stop_words( ws ):
    return [ w for w in ws if re.match( single, w ) is None and re.match( pair, w ) is None ]

#### トークナイズする

In [11]:
csv['wakachi'] = csv.apply( lambda x: wakachi( x['review'] ), axis=1 )

In [12]:
csv['words'] = csv.apply( lambda x: stop_words_set( x['wakachi'] ), axis=1 )

In [23]:
csv[:5]

,code,meigara,review,datetime,wakachi,words,tokens
0,4,鰊御殿 山田錦,4合瓶900円で安売りだったので買ってみました。あまり期待してなかったのですが、わりと普通に...,2014-09-14 10:32:46,"[4, 合, 瓶, 900, 円, で, 安売り, だっ, た, ので, 買っ, て, み,...","{あまり, 合, 900, 円, 旨かっ, ほどよく, なかっ, 買っ, 瓶, 普通, アル...","[合, 瓶, 900, 円, 安売り, 買っ, あまり, 期待, なかっ, 普通, 旨かっ,..."
1,4,北の誉 鰊御殿 純米大吟醸酒,山田錦100％使用。 フルーティな香りと、あっさりした呑み口、 コクのある味わい。 文句の...,2014-05-23 21:35:40,"[山田, 錦, 100, ％, 使用, 。, フルーティ, な, 香り, と, 、, あっさ...","{山田, 大, 文句, 錦, 純, 酒, 米, 香り, 高い, 口, 残念, ％, 味わい,...","[山田, 錦, 100, ％, 使用, フルーティ, 香り, あっさり, 呑み, 口, 味わ..."
2,4,純米米しぼり,アルコール度数14-15。精米歩合70％。 香りマイルド、深いコク。 オーソドックスな味わ...,2013-08-28 23:19:33,"[アルコール, 度数, 14, -, 15, 。, 精米, 歩合, 70, ％, 。, 香り...","{苦味, どんな, ％, 味わい, オーソドックス, マイルド, 料理, 合う, 嫌み, 1...","[アルコール, 度数, 14, 15, 精米, 歩合, 70, ％, 香り, マイルド, 深..."
3,4,白ラベル,安いが美味いと思う、ぬる燗か常温がいいね、吟醸酒をブレンドして造ってるて噂があるけどどうでし...,2009-07-07 00:43:14,"[安い, が, 美味い, と, 思う, 、, ぬる, 燗, か, 常温, が, いい, ね,...","{安い, 酒, 造っ, 燗, 美味い, 噂, 思う, 常温, でしょ, 吟醸, ブレンド}","[安い, 美味い, 思う, 燗, 常温, 吟醸, 酒, ブレンド, 造っ, 噂, でしょ]"
4,4,純米吟醸『涼しぼり』,近所の酒屋で購入。 「夏のにごり酒」という謳い文句通り、爽やかで呑みやすい。 香りもいいけど...,2008-06-16 23:37:21,"[近所, の, 酒屋, で, 購入, 。, 「, 夏, の, にごり酒, 」, という, 謳...","{にごり酒, 香り, 夏, 通り, 爽やか, なかっ, という, オンザロック, 酒屋, 呑...","[近所, 酒屋, 購入, 夏, にごり酒, という, 謳い文句, 通り, 爽やか, 呑み, ..."


In [14]:
from functools import reduce

In [15]:
wdict = reduce( lambda s, x: s | x, list(csv.words) )
#wdict = reduce( lambda a, x: a + x, [1,2,3,4,5] )
len(wdict)

12336

In [16]:
csv['tokens'] = csv.apply( lambda x: stop_words( x['wakachi'] ), axis=1 )

#### 単語がどのくらい出てくるか確認してみる

In [17]:
def wc( a, items ):
    for x in items:
        if x not in a:
            a[x] = 1
        else:
            a[x] += 1
    return a

In [18]:
wc = reduce( lambda a, x: wc( a, x ), list(csv.tokens), dict() )

In [19]:
wdf = pd.DataFrame( list(wc.items()), columns=['word', 'count'] )

In [20]:
sorted_wdf = wdf.sort_values( 'count', ascending=False )

In [21]:
sorted_wdf[:20]

,word,count
10943,酒,5995
930,米,2148
2378,感じ,2039
12109,香り,2005
8656,味,1918
8389,飲み,1498
6519,純,1276
8917,吟醸,1267
8975,飲ん,1103
2632,味わい,1011


In [29]:
dictionary = corpora.Dictionary( list( csv.tokens ) )
print( dictionary )
print( dictionary.token2id )

Dictionary(12336 unique tokens: ['シェリー', '浸し', '国分', '夏', '華やかさ']...)
{'シェリー': 5484, '浸し': 2235, '国分': 3593, '夏': 71, '華やかさ': 1469, 'kg': 10458, '乃': 1464, '梅乃宿': 10782, 'パワーアップ': 1848, '夕顔': 9465, '龍泉': 2141, '朝酒': 9187, '言わば': 8789, '菅原': 9905, '逞しく': 4414, '奥ゆかし': 6791, 'ねじ伏せる': 3521, 'にとって': 975, '上品': 211, '弁天': 4369, '暖房': 9686, '先月': 6355, '65': 1647, '寒空': 6380, '閑静': 11895, '福無量': 8891, '付け方': 4842, '勿体なく': 9910, '旨み': 208, '騙し': 2642, '中頃': 4978, '味わい深': 8514, 'ＮＧ': 5517, '誇る': 9076, '惹か': 2789, '程々': 9061, '硬': 2428, '懲り': 12104, '体調': 5919, 'とても': 240, '重く': 1597, 'ハーゲンダッツ': 5526, '歴戦': 6853, 'ごめんなさい': 1202, '喪主': 9483, 'にぶる': 10025, '覆っ': 2796, 'すると': 2903, '点前': 10370, 'やかん': 8022, 'こわばっ': 8429, '発売': 2905, 'じゃん': 8998, 'メロン': 704, '奮発': 2387, '楽天': 7983, '出': 463, '小売り': 8254, '裏': 2672, '甘酸っぱ': 2572, '間違い': 1274, 'おそるおそる': 4971, '平面': 2837, 'ぁっとお': 10127, '出品': 3158, 'ﾄｺ': 8450, '増える': 863, 'to': 2851, 'Really': 8236, '不向き': 7957, '輝': 12151, '出国': 10700, 'そして': 1038, '

In [40]:
dictionary.filter_extremes(no_below=20, no_above=0.3)
# no_berow: 使われてる文章がno_berow個以下の単語無視
# no_above: 使われてる文章の割合がno_above以上の場合無視
print( dictionary )

Dictionary(1095 unique tokens: ['もっと', '楽しめる', '苦味', 'バナナ', '協会']...)


### 2. 特徴ベクトルをつくる

In [51]:
tmp = dictionary.doc2bow( ['合', '瓶', '合', '円', '安売り', '買っ', 'あまり', '期待', 'なかっ', '普通', '酸味'] )
tmp

[(259, 1),
 (290, 1),
 (376, 2),
 (382, 1),
 (524, 1),
 (807, 1),
 (995, 1),
 (996, 1),
 (1084, 1)]

In [30]:
corpus = [ dictionary.doc2bow( doc ) for doc in list(csv.tokens) ]

In [46]:
terms = len(dictionary)
dense = list(matutils.corpus2dense([tmp], num_terms = terms ).T[0])
print( dense )

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [47]:
terms = len(dictionary)
T = [ matutils.corpus2dense([dictionary.doc2bow( doc )], num_terms = terms ).T[0] for doc in list(csv.tokens) ]
print ( T )

[array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32), array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32), array([ 0.,  0.,  1., ...,  0.,  0.,  0.], dtype=float32), array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32), array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32), array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32), array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32), array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32), array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32), array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32), array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32), array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32), array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32), array([ 0.,  0.,  0., ...,  0.,  0.,  1.], dtype=float32), array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32), array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32), array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float3

In [48]:
import numpy as np

In [50]:
np.array( T )

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

### 3. クラスタリングする

In [52]:
from sklearn.cluster import KMeans

In [57]:
features = np.array( T )
kmeans_model = KMeans( n_clusters=5, random_state=10 ).fit( features )

In [58]:
# 分類先となったラベルを取得する
labels = kmeans_model.labels_

In [56]:
# ラベル と 銘柄を表示する
for label, code, meigara in zip(labels, list(csv.code), list(csv.meigara)):
    print(label, code, meigara)

3 4 鰊御殿　山田錦
2 4 北の誉　鰊御殿　純米大吟醸酒
3 4 純米米しぼり
3 4 白ラベル
3 4 純米吟醸『涼しぼり』
0 4 純米生原酒「しぼりたて」
0 4 純米酒｢吟心｣
3 1449 ほんとに危険です！
3 1449 危険です！
3 1449 淡麗辛口
0 2758 純米吟醸
3 5 蔵元限定　純米生酒　
3 5 千歳鶴　純米にごり新酒
3 5 千歳鶴　春のしぼりたて　特別純米酒
0 5 純米吟醸
2 5 「ぱり」という酒がありました。
3 6 北のろまん
0 6 まる田（特別純米）
3 1891 純米吟醸しずくどり
3 150 大吟醸　一夜雫
3 150 ウ、うまいんです
3 150 すっとした飲み心地です
3 150 居酒屋北海道で一番の酒
1 3 純米酒
1 3 国士無双　純米大吟醸　北海道限定　彗星
3 3 国士無双　純米大吟醸　北海道限定
3 3 国士無双　純米酒
1 3 純米　国士無双
3 3 国士無双大吟醸
3 3 北海道の日本酒
3 3 大人味
2 2 金滴　彗星純米大吟醸　生
2 2 特別本醸造原酒
3 2 北海道へ行くと探します。
3 2 金冠
3 9 國稀 特別純米酒　五百万石　55％
3 9 國稀 北海 にごり酒　ススキノにて
3 9 北海鬼ころし
3 9 原酒のにごり
3 8 癖のない
3 7 純米大吟醸
1 7 普通酒
0 7 特別純米酒「木綿屋」
1 7 男山 原酒
3 7 男山 特別純米 北海道限定　美山
2 7 男山　生酒元（きもと）　純米酒　
1 7 北海男山 純米大吟醸
1 1452 北の酒
1 22 南部蔵 富貴 特別純米酒
2 24 陸奥八仙純米大吟醸華想い50
0 24 黒ラベル 純米吟醸 火入
3 24 純米吟醸無濾過生直汲み（黒ラベル）
1 24 純米芳醇辛口　無濾過生原酒
3 24 特別純米　中汲み無濾過生原酒
0 24 ISARIBI　特別純米　無濾過生詰
1 24 おいしかった。。。。
3 24 特別純米　無濾過原酒瓶火　ひやおろし
1 24 特別純米無濾過生原酒中汲み
2 24 出張で。
3 158 純米生貯蔵原酒　氷清
2 19 菊乃井　ひやおろし　純米生原酒
0 1665 飯田橋の青森アンテナショップで購入
3 155 食前酒にドオゥ?
0 14 純米吟醸　ひやおろし
3 14 純米吟醸

In [59]:
sake = pd.read_csv( '~/work/sake/sake.csv' )

In [60]:
sake[:5]

,code,meigara,kana,kuramoto,ken,shi,address
0,1235,京極,きょうごく,二世古酒造,北海道,虻田郡,北海道虻田郡倶知安町旭47
1,1234,二世古,にせこ,二世古酒造,北海道,虻田郡,北海道虻田郡倶知安町旭47
2,1447,雪の花,ゆきのはな,雪の花酒造,北海道,小樽市,北海道小樽市
3,1446,小樽港,おたるこう,雪の花酒造,北海道,小樽市,北海道小樽市
4,4,北の誉,きたのほまれ,北の誉酒造,北海道,小樽市,北海道小樽市奥沢1-21-15


In [65]:
joined = csv.join( sake, on='code', how='left', rsuffix='_s' )

In [66]:
joined[:5]

,code,meigara,review,datetime,wakachi,words,tokens,code_s,meigara_s,kana,kuramoto,ken,shi,address
0,4,鰊御殿 山田錦,4合瓶900円で安売りだったので買ってみました。あまり期待してなかったのですが、わりと普通に...,2014-09-14 10:32:46,"[4, 合, 瓶, 900, 円, で, 安売り, だっ, た, ので, 買っ, て, み,...","{あまり, 合, 900, 円, 旨かっ, ほどよく, なかっ, 買っ, 瓶, 普通, アル...","[合, 瓶, 900, 円, 安売り, 買っ, あまり, 期待, なかっ, 普通, 旨かっ,...",4,北の誉,きたのほまれ,北の誉酒造,北海道,小樽市,北海道小樽市奥沢1-21-15
1,4,北の誉 鰊御殿 純米大吟醸酒,山田錦100％使用。 フルーティな香りと、あっさりした呑み口、 コクのある味わい。 文句の...,2014-05-23 21:35:40,"[山田, 錦, 100, ％, 使用, 。, フルーティ, な, 香り, と, 、, あっさ...","{山田, 大, 文句, 錦, 純, 酒, 米, 香り, 高い, 口, 残念, ％, 味わい,...","[山田, 錦, 100, ％, 使用, フルーティ, 香り, あっさり, 呑み, 口, 味わ...",4,北の誉,きたのほまれ,北の誉酒造,北海道,小樽市,北海道小樽市奥沢1-21-15
2,4,純米米しぼり,アルコール度数14-15。精米歩合70％。 香りマイルド、深いコク。 オーソドックスな味わ...,2013-08-28 23:19:33,"[アルコール, 度数, 14, -, 15, 。, 精米, 歩合, 70, ％, 。, 香り...","{苦味, どんな, ％, 味わい, オーソドックス, マイルド, 料理, 合う, 嫌み, 1...","[アルコール, 度数, 14, 15, 精米, 歩合, 70, ％, 香り, マイルド, 深...",4,北の誉,きたのほまれ,北の誉酒造,北海道,小樽市,北海道小樽市奥沢1-21-15
3,4,白ラベル,安いが美味いと思う、ぬる燗か常温がいいね、吟醸酒をブレンドして造ってるて噂があるけどどうでし...,2009-07-07 00:43:14,"[安い, が, 美味い, と, 思う, 、, ぬる, 燗, か, 常温, が, いい, ね,...","{安い, 酒, 造っ, 燗, 美味い, 噂, 思う, 常温, でしょ, 吟醸, ブレンド}","[安い, 美味い, 思う, 燗, 常温, 吟醸, 酒, ブレンド, 造っ, 噂, でしょ]",4,北の誉,きたのほまれ,北の誉酒造,北海道,小樽市,北海道小樽市奥沢1-21-15
4,4,純米吟醸『涼しぼり』,近所の酒屋で購入。 「夏のにごり酒」という謳い文句通り、爽やかで呑みやすい。 香りもいいけど...,2008-06-16 23:37:21,"[近所, の, 酒屋, で, 購入, 。, 「, 夏, の, にごり酒, 」, という, 謳...","{にごり酒, 香り, 夏, 通り, 爽やか, なかっ, という, オンザロック, 酒屋, 呑...","[近所, 酒屋, 購入, 夏, にごり酒, という, 謳い文句, 通り, 爽やか, 呑み, ...",4,北の誉,きたのほまれ,北の誉酒造,北海道,小樽市,北海道小樽市奥沢1-21-15


In [67]:
# ラベル と 銘柄を表示する
for label, meigara, sub in zip(labels, list(joined.meigara_s), list(joined.meigara)):
    print(label, meigara, sub)

0 北の誉 鰊御殿　山田錦
1 北の誉 北の誉　鰊御殿　純米大吟醸酒
0 北の誉 純米米しぼり
0 北の誉 白ラベル
0 北の誉 純米吟醸『涼しぼり』
4 北の誉 純米生原酒「しぼりたて」
4 北の誉 純米酒｢吟心｣
0 富士正 ほんとに危険です！
3 富士正 危険です！
0 富士正 淡麗辛口
4 宗政 純米吟醸
0 鬼丸 蔵元限定　純米生酒　
0 鬼丸 千歳鶴　純米にごり新酒
0 鬼丸 千歳鶴　春のしぼりたて　特別純米酒
4 鬼丸 純米吟醸
1 鬼丸 「ぱり」という酒がありました。
0 飲ん米 北のろまん
4 飲ん米 まる田（特別純米）
0 池雲 純米吟醸しずくどり
0 酔楽天 大吟醸　一夜雫
0 酔楽天 ウ、うまいんです
0 酔楽天 すっとした飲み心地です
3 酔楽天 居酒屋北海道で一番の酒
2 小樽港 純米酒
2 小樽港 国士無双　純米大吟醸　北海道限定　彗星
3 小樽港 国士無双　純米大吟醸　北海道限定
0 小樽港 国士無双　純米酒
2 小樽港 純米　国士無双
3 小樽港 国士無双大吟醸
0 小樽港 北海道の日本酒
3 小樽港 大人味
1 雪の花 金滴　彗星純米大吟醸　生
1 雪の花 特別本醸造原酒
3 雪の花 北海道へ行くと探します。
0 雪の花 金冠
0 北宝 國稀 特別純米酒　五百万石　55％
3 北宝 國稀 北海 にごり酒　ススキノにて
0 北宝 北海鬼ころし
0 北宝 原酒のにごり
0 小樽の女 癖のない
0 宝川 純米大吟醸
2 宝川 普通酒
4 宝川 特別純米酒「木綿屋」
2 宝川 男山 原酒
3 宝川 男山 特別純米 北海道限定　美山
3 宝川 男山　生酒元（きもと）　純米酒　
2 宝川 北海男山 純米大吟醸
2 富士山 北の酒
2 国士無双 南部蔵 富貴 特別純米酒
1 国稀 陸奥八仙純米大吟醸華想い50
4 国稀 黒ラベル 純米吟醸 火入
3 国稀 純米吟醸無濾過生直汲み（黒ラベル）
2 国稀 純米芳醇辛口　無濾過生原酒
0 国稀 特別純米　中汲み無濾過生原酒
4 国稀 ISARIBI　特別純米　無濾過生詰
3 国稀 おいしかった。。。。
0 国稀 特別純米　無濾過原酒瓶火　ひやおろし
2 国稀 特別純米無濾過生原酒中汲み
1 国稀 出張で。
0 両関 純米生貯蔵原酒　氷清
1 気まぐれ 菊乃井　ひやおろし　純米生原酒
4 東洋自慢 飯田